In [1]:
# where is it ?

import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import TargetEncoder, StandardScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from catboost import CatBoostRegressor

In [12]:
train_df_full = pd.read_csv('train.csv')#'/kaggle/input/playground-series-s4e9/train.csv')
test_df_full = pd.read_csv('test.csv')#'/kaggle/input/playground-series-s4e9/test.csv')

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pandas as pd

# Define PyTorch model
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        return self.fc3(x)

# Function to preprocess data
def preprocess_data(X, y=None, target_encoder=None, scaler=None):
    # no need to bin here in the NN 
    #X = X.assign(mileage=X['milage'] // 100).drop(columns='milage')


    # if you want to remove the id
    if 'id' in X.columns:
        X = X.drop(columns=['id'])

    # Apply TargetEncoder and StandardScaler
    if target_encoder is not None:
        if y is not None:
            X = target_encoder.fit_transform(X, y)
        else:
            X = target_encoder.transform(X)
    if scaler is not None:
        if y is not None:
            X = scaler.fit_transform(X)
        else:
            X = scaler.transform(X)
    return X

# Function to train PyTorch model
def train_pytorch_model(X_train, y_train, input_size, epochs=20, batch_size=64):
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

    # Create DataLoader
    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # Initialize PyTorch model, loss, and optimizer
    model = SimpleNN(input_size=input_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    model.train()
    for epoch in range(epochs):
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
    return model

# Function to evaluate PyTorch model
def evaluate_pytorch_model(model, X_test, y_test):
    # Convert test data to PyTorch tensors
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        rmse = torch.sqrt(torch.mean((predictions - y_test_tensor) ** 2)).item()
    return rmse

# Function to train and evaluate Ridge regression
def train_and_evaluate_ridge(X_train, y_train, X_test, y_test):
    ridge_model = Ridge(alpha=1.0)
    ridge_model.fit(X_train, y_train)
    predictions = ridge_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    return rmse

# Function for cross-validation loop
def cross_validate(X, y, target_encoder, scaler, n_splits=5, n_repeats=1, epochs=20, batch_size=64):
    kfold = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=SEED)
    rmse_scores_nn = []
    rmse_scores_ridge = []

    for train_index, test_index in kfold.split(X):
        print('a fold')
        # Split the data
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Preprocess the data
        X_train_scaled = preprocess_data(X_train, y_train, target_encoder, scaler)
        X_test_scaled = preprocess_data(X_test, target_encoder=target_encoder, scaler=scaler)

        # Train and evaluate PyTorch model
        model = train_pytorch_model(X_train_scaled, y_train, input_size=X_train_scaled.shape[1], epochs=epochs, batch_size=batch_size)
        rmse_nn = evaluate_pytorch_model(model, X_test_scaled, y_test)
        rmse_scores_nn.append(rmse_nn)

        # Train and evaluate Ridge regression
        rmse_ridge = train_and_evaluate_ridge(X_train_scaled, y_train, X_test_scaled, y_test)
        rmse_scores_ridge.append(rmse_ridge)

    print(f'PyTorch Neural Network CV RMSE score: {np.mean(rmse_scores_nn):.5f} ± {np.std(rmse_scores_nn):.5f}')
    print(f'Ridge Regression CV RMSE score: {np.mean(rmse_scores_ridge):.5f} ± {np.std(rmse_scores_ridge):.5f}')


# Prepare data
SEED = 4
DEV_FRACTION = .5
df_dev = train_df_full.sample(frac=DEV_FRACTION, random_state=SEED)

print(df_dev.shape)

# Separate features (X) and target (y)
X = df_dev.drop(columns='price')
y = df_dev['price']


# Use only the 12 features for now
#X = X.iloc[:, :12]

# Cross-validation setup
kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=SEED)

# Initialize TargetEncoder and StandardScaler
target_encoder = TargetEncoder(random_state=SEED, target_type='continuous').set_output(transform='pandas')
scaler = StandardScaler()


# Run cross-validation
cross_validate(X, y, target_encoder, scaler)


(94266, 13)
a fold
(75412, 11)
(18854, 11)
a fold
(75413, 11)
(18853, 11)
a fold
(75413, 11)
(18853, 11)
a fold
(75413, 11)
(18853, 11)
a fold
(75413, 11)
(18853, 11)
PyTorch Neural Network CV RMSE score: 72241.20938 ± 4056.83900
Ridge Regression CV RMSE score: 72302.20180 ± 4056.82044


# producing the output

In [29]:
# Main function to train on full data and make a submission
def train_full_model_and_submit(train_df, test_df, target_encoder, scaler, epochs=20, batch_size=64):
    # Separate features and target
    X_train = train_df.drop(columns='price')
    y_train = train_df['price']
    X_test = test_df
    
    # Preprocess the data
    X_train_scaled = preprocess_data(X_train, y_train, target_encoder, scaler)

    # Train the PyTorch model on the full training data
    model = train_pytorch_model(X_train_scaled, y_train, input_size=X_train_scaled.shape[1], epochs=epochs, batch_size=batch_size)

    # Prepare the test data and make predictions
    X_test_scaled = preprocess_data(X_test, target_encoder=target_encoder, scaler=scaler)

    # Make predictions
    model.eval()
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    with torch.no_grad():
        predictions = model(X_test_tensor).numpy().flatten()

    # Create submission DataFrame
    submission = pd.DataFrame({
        'id': test_df['id'],
        'price': predictions
    })

    # Save submission to CSV
    submission.to_csv('submission8_pytorchNOid.csv', index=False)


# Train the full model and create a submission
train_full_model_and_submit(train_df_full, test_df_full, target_encoder, scaler)

(188533, 11)
(125690, 11)
